### Introduction
Climate change presents a profound challenge to agricultural productivity worldwide. The frequency of extreme weather events, variability in rainfall patterns, and shifts in temperature are all contributing to increasing uncertainty in agricultural outputs. In regions heavily reliant on agriculture, such as many parts of Kenya, the impact of these changes can have severe implications not just for local economies, but also for food security and overall societal stability.

### Project Overview
This project aims to leverage extensive meteorological and agricultural data to analyze the impacts of climate change on crop yields. By examining historical data and identifying patterns and trends, the project seeks to forecast future conditions and provide actionable insights that can help mitigate the adverse effects of climate change on agriculture.

#### Business Problem
The primary challenge addressed by this project is the need for enhanced predictive capabilities that can accurately forecast the impacts of various climatic factors on crop yields. Current agricultural planning and operations often rely on traditional knowledge and simplistic predictive models that fail to account for the increasing complexity and variability introduced by climate change.

#### Stakeholders
Farmers and Agricultural Cooperatives: Direct beneficiaries who require accurate predictions to make informed decisions about crop planning and resource allocation.
Government Agencies: Need insights to formulate effective agricultural policies and climate adaptation strategies.
Agricultural Researchers and Academics: Interested in data-driven insights to enhance scientific understanding and develop more resilient agricultural practices.
NGOs and International Aid Organizations: Require accurate forecasts to better target their support and interventions in regions most affected by climate change.
#### Objectives
Analyze Historical Data: Understand how temperature, rainfall, and other factors have historically affected crop yields.
Develop Predictive Models: Create robust models that can predict crop yields based on various climatic inputs.
Provide Decision Support: Offer actionable recommendations that can help stakeholders adjust to changing conditions more effectively.
Enhance Climate Resilience: Contribute to building a more climate-resilient agricultural sector by integrating advanced analytics into farming practices.
##### Research Questions
How do variations in temperature and rainfall correlate with changes in crop yields?
Which crops are most vulnerable to changes in specific climatic factors?
Can we predict crop yield changes based on forecasted weather patterns?
What are the most effective strategies for mitigating adverse climate impacts on agriculture?

In [ ]:
!pip install pandas


In [ ]:
import pandas as pd

In [ ]:
yield_csv = "/content/climate-ds.csv"

In [ ]:
yield_df = pd.read_csv(yield_csv)

In [ ]:
yield_df.head()

In [ ]:
yield_df.info()

In [ ]:
# Dropping the 'Unnamed: 0' column from the DataFrame
yield_df = yield_df.drop(columns=['Unnamed: 0'])

# Verifying that the column has been dropped
print(yield_df.head())


In [ ]:
yield_df.shape

In [ ]:
yield_df.describe().T

In [ ]:
yield_df['Area'].value_counts()

In [ ]:
# india is the most frequent ,followed by Brazil and Mexico


In [ ]:
yield_df['Item'].value_counts()

In [ ]:
# Potatoes are the most popular crop followed by Maize and Wheat

In [ ]:
yield_df.isna().sum()

In [ ]:
yield_df.duplicated().sum()

In [ ]:
# Display some duplicate rows to understand their nature
print(yield_df[yield_df.duplicated(keep=False)].sort_values(by=['Area', 'Year', 'Item']).head())


In [ ]:
# Remove duplicates considering specific columns that define a unique entry
yield_df = yield_df.drop_duplicates(subset=['Area', 'Item', 'Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp'])

# Verify the removal by checking for duplicates again
print("Remaining duplicates:", yield_df.duplicated(subset=['Area', 'Item', 'Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']).sum())


In [ ]:
# Check the new shape of the DataFrame
print("Updated DataFrame shape:", yield_df.shape)


In [ ]:
yield_df.info()

In [ ]:
yield_df['pesticides_tonnes'].value_counts()

In [ ]:
yield_df['average_rain_fall_mm_per_year'].value_counts()

In [ ]:
# Group by 'Area', sum the yields, and find the top 10 countries with the highest total yields
top_countries_by_yield = yield_df.groupby('Area', sort=True)['hg/ha_yield'].sum().nlargest(10)

# Display the result
print(top_countries_by_yield)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the result
top_countries_by_yield.plot(kind='bar', color='skyblue')
plt.title('Top 10 Countries by Total Crop Yield')
plt.xlabel('Country')
plt.ylabel('Total Crop Yield (hg/ha)')
plt.xticks(rotation=45)
plt.show()


 india is the highest crop yeilds followed by Brazil and mexico

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histograms for all numeric data
yield_df.hist(bins=15, figsize=(15, 10))
plt.suptitle('Histograms of All Variables')
plt.show()

# Box plots to identify outliers
for column in ['hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=yield_df[column])
    plt.title(f'Box Plot of {column}')
    plt.show()


The average yeild is between 0 and 100000 hg/ha ,the average rainfall is between 500 and 1500,the average pesticide used is between 0-50000tons and the average temperature is between 25 and 28C

there is also presence of outlier in the data and therefore some transformations may be needed before modeling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Adding 1 to ensure there are no log(0) issues
yield_df['log_hg/ha_yield'] = np.log1p(yield_df['hg/ha_yield'])


# Check for any zero or negative values in rainfall if you want to include it
# yield_df['log_average_rain_fall_mm_per_year'] = np.log1p(yield_df['average_rain_fall_mm_per_year'])

# Updated list of variables to plot
variables_transformed = ['log_hg/ha_yield', 'pesticides_tonnes', 'avg_temp','average_rain_fall_mm_per_year']

# Replotting histograms to check improvements
yield_df[variables_transformed].hist(bins=15, figsize=(15, 5), layout=(1, 4))
plt.suptitle('Histograms of Log-Transformed Variables')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# List of transformed variables to plot
variables_transformed = ['log_hg/ha_yield', 'pesticides_tonnes', 'avg_temp']

# Set up the matplotlib figure
plt.figure(figsize=(18, 12))

# Number of rows needed for subplots
num_vars = len(variables_transformed)
rows = num_vars

# Loop through the list of variables and create a subplot for each histogram and box plot
for i, var in enumerate(variables_transformed, 1):
    plt.subplot(rows, 2, 2 * i - 1)  # Odd index for histograms
    sns.histplot(yield_df[var], kde=True)
    plt.title(f'Histogram of {var}')

    plt.subplot(rows, 2, 2 * i)  # Even index for box plots
    sns.boxplot(x=yield_df[var])
    plt.title(f'Box Plot of {var}')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Define a function to remove outliers based on IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to the log-transformed yield
cleaned_df = remove_outliers(yield_df, 'log_hg/ha_yield')

# Plotting to verify removal of outliers
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(cleaned_df['log_hg/ha_yield'], kde=True)
plt.title('Histogram After Removing Outliers')
plt.subplot(1, 2, 2)
sns.boxplot(x=cleaned_df['log_hg/ha_yield'])
plt.title('Box Plot After Removing Outliers')
plt.show()


In [ ]:
# Compute the correlation matrix
corr_matrix = cleaned_df[['hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp','log_hg/ha_yield']].corr()


In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .8})

# Adding title and labels
plt.title('Correlation Matrix of Crop Data')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()


The correlation between the log-transformed crop yield (log_hg/ha_yield) and other variables like rainfall, temperature, and pesticide usage appears relatively low.
The strongest correlations involving log_hg/ha_yield are with the original yield values, which is expected due to their direct mathematical relationship.

In [ ]:
# Pairplot to explore the relationships between all continuous variables
sns.pairplot(cleaned_df[['log_hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']])
plt.show()


In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.shape

In [ ]:
cleaned_df.info()

In [ ]:

X = cleaned_df.drop(columns=["hg/ha_yield", "log_hg/ha_yield"])
X

In [ ]:
print(X.columns)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Assuming 'X' is your features DataFrame and 'cleaned_df' contains the target variable 'log_hg/ha_yield'

# Updating the list of numeric and categorical features
numeric_features = ['avg_temp', 'average_rain_fall_mm_per_year', 'pesticides_tonnes']
categorical_features = ['Area', 'Item']  # Added Item as a categorical feature

# Setting up the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Applying the preprocessing to the DataFrame
X_preprocessed = preprocessor.fit_transform(X)

# Print categories used during training
print(preprocessor.named_transformers_['cat'].categories_)

# Target variable remains the same
y = cleaned_df['log_hg/ha_yield']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.3, random_state=42)

# Get feature names after preprocessing
numeric_feature_names = numeric_features
categorical_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)

# Combine all feature names
feature_names = np.concatenate([numeric_feature_names, categorical_feature_names])

# Ensure the lengths match for verification
print(f"Length of feature_names: {len(feature_names)}")




### baseline Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression Model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

# Evaluate the model
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)
print(f"Linear Regression MSE: {mse_linear}")
print(f"Linear Regression R^2: {r2_linear}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate residuals
residuals = y_test - y_pred_linear

# Plotting residuals
plt.figure(figsize=(12, 6))

# Residuals Scatter Plot
plt.subplot(1, 2, 1)
plt.scatter(y_pred_linear, residuals, alpha=0.5)
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='--')

# Histogram of Residuals
plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.show()


In [ ]:
import statsmodels.api as sm

plt.figure(figsize=(6, 6))
sm.qqplot(residuals, line='s')
plt.title('Q-Q Plot of Residuals')
plt.show()


In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_pred_linear, y_test, alpha=0.5)
plt.title('Observed vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Observed Values')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')  # Diagonal line for reference
plt.show()


In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(linear_model, X_preprocessed, y, cv=5, scoring='r2')

# Print the cross-validation scores
print("Cross-validated R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Adding polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_train)

# Refit the linear model with polynomial features
linear_model_poly = LinearRegression()
linear_model_poly.fit(X_poly, y_train)
y_pred_poly = linear_model_poly.predict(poly.transform(X_test))

# Evaluate the new model
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print(f"Polynomial Regression MSE: {mse_poly}")
print(f"Polynomial Regression R²: {r2_poly}")


### regulization

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

# Lasso Regression
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

# Elastic Net Regression
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_model.fit(X_train, y_train)
y_pred_elastic = elastic_model.predict(X_test)

# Evaluation
print("Ridge R²:", r2_score(y_test, y_pred_ridge))
print("Lasso R²:", r2_score(y_test, y_pred_lasso))
print("Elastic Net R²:", r2_score(y_test, y_pred_elastic))


In [ ]:
from sklearn.model_selection import cross_val_score


ridge_cv_scores = cross_val_score(ridge_model, X_preprocessed, y, cv=5, scoring='r2')
print("Ridge CV R² scores:", ridge_cv_scores)
print("Ridge Mean CV R²:", np.mean(ridge_cv_scores))


### Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn import tree

# Assuming that the preprocessing steps are done and X_train, X_test, y_train, y_test are defined
# Assuming preprocessor is the ColumnTransformer used for preprocessing

# Initialize the DecisionTreeRegressor
tree_regressor = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the model on the training data
tree_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred_tree = tree_regressor.predict(X_test)

# Evaluate the model
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)
print(f"Decision Tree MSE: {mse_tree}")
print(f"Decision Tree R²: {r2_tree}")

# Extract feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Visualize the tree
plt.figure(figsize=(20, 10))
tree.plot_tree(tree_regressor, filled=True, feature_names=feature_names, rounded=True, fontsize=12)
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# Define the model
tree_regressor = DecisionTreeRegressor(random_state=42)

# Create a parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=tree_regressor, param_grid=param_grid, cv=5, scoring='r2', verbose=1, n_jobs=-1)

# Fit to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validated R² score: ", grid_search.best_score_)

# Use the best estimator to make predictions
best_tree = grid_search.best_estimator_
y_pred_best_tree = best_tree.predict(X_test)

# Evaluate the best model
mse_best_tree = mean_squared_error(y_test, y_pred_best_tree)
r2_best_tree = r2_score(y_test, y_pred_best_tree)
print(f"Optimized Decision Tree MSE: {mse_best_tree}")
print(f"Optimized Decision Tree R²: {r2_best_tree}")


### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Fit the model on the training data
random_forest.fit(X_train, y_train)

# Predict on the test data
y_pred_rf = random_forest.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest MSE: {mse_rf}")
print(f"Random Forest R²: {r2_rf}")

# Feature Importance
feature_importances = random_forest.feature_importances_
# Assuming we have feature names in a list called 'feature_names'
feature_names = preprocessor.get_feature_names_out()
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)
print(importance_df.head())


### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Set up parameter grid to tune the hyperparameters
param_grid = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [None, 10, 20],  # Maximum depth of trees
    'min_samples_leaf': [1, 2, 4]  # Minimum samples at a leaf node
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best R²:", grid_search.best_score_)


### Gradient Boosting Regression

In [ ]:


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Train the Gradient Boosting model
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred_gb = gb_regressor.predict(X_test)

# Evaluate the model
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
print(f"Gradient Boosting MSE: {mse_gb}")
print(f"Gradient Boosting R²: {r2_gb}")

# Ensure the lengths match
print(f"Length of feature_importances_gb: {len(gb_regressor.feature_importances_)}")

# If lengths match, create DataFrame
if len(feature_names) == len(gb_regressor.feature_importances_):
    # Create DataFrame
    importance_df_gb = pd.DataFrame({'Feature': feature_names, 'Importance': gb_regressor.feature_importances_}).sort_values(by='Importance', ascending=False)
    print(importance_df_gb.head())
else:
    print("The lengths of feature_names and feature_importances_gb do not match")





In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.01]
}

grid_search_gb = GridSearchCV(estimator=gb_regressor, param_grid=param_grid_gb, cv=3, scoring='r2', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)
print("Best parameters for Gradient Boosting:", grid_search_gb.best_params_)
print("Best R² for Gradient Boosting:", grid_search_gb.best_score_)


### Support Vector Regression (SVR)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the SVR
svr_regressor = SVR(kernel='rbf', C=1.0, epsilon=0.1)  # RBF kernel, adjust C and epsilon as needed

# Fit the model on the training data
svr_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred_svr = svr_regressor.predict(X_test)

# Evaluate the model
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)
print(f"SVR MSE: {mse_svr}")
print(f"SVR R²: {r2_svr}")


In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameter grid for SVR
param_grid_svr = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'epsilon': [0.01, 0.1, 0.2],
    'kernel': ['rbf']
}

grid_search_svr = GridSearchCV(estimator=svr_regressor, param_grid=param_grid_svr, cv=3, scoring='r2', verbose=1, n_jobs=4)
grid_search_svr.fit(X_train, y_train)

# Output the best parameters and the best score
print("Best parameters for SVR:", grid_search_svr.best_params_)
print("Best R² for SVR:", grid_search_svr.best_score_)


### Neural Network Regression with MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the MLPRegressor
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001,
                             batch_size='auto', learning_rate='constant', learning_rate_init=0.001, max_iter=200,
                             random_state=42, verbose=True, n_iter_no_change=10, tol=0.0001)

# Fit the model on the training data
mlp_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred_mlp = mlp_regressor.predict(X_test)

# Evaluate the model
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)
print(f"MLP MSE: {mse_mlp}")
print(f"MLP R²: {r2_mlp}")


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

# Initialize the MLPRegressor with very basic settings
mlp_regressor = MLPRegressor(max_iter=300, random_state=42, solver='lbfgs')  # Changed solver to 'lbfgs' for stability

# Define an even more restricted parameter grid
param_grid_mlp = {
    'hidden_layer_sizes': [(50,)],  # Single layer with fewer neurons
    'activation': ['tanh'],  # Only one activation function
}

# Set up GridSearchCV with no parallel jobs
grid_search_mlp = GridSearchCV(estimator=mlp_regressor, param_grid=param_grid_mlp, cv=3, scoring='r2', verbose=1, n_jobs=1)

# Fit GridSearchCV

grid_search_mlp.fit(X_train, y_train)
print("Best parameters for MLP:", grid_search_mlp.best_params_)
print("Best R² for MLP:", grid_search_mlp.best_score_)



### Time Series Analysis

In [ ]:
cleaned_df.head()

In [ ]:
N_cleaned_df=yield_df.copy()

In [ ]:
# Convert 'Year' to a datetime type if necessary and sort the data
N_cleaned_df['Year'] = pd.to_datetime(N_cleaned_df['Year'], format='%Y')
N_cleaned_df.sort_values('Year', inplace=True)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 10))

# Using the cleaned and transformed data for crop yield
plt.subplot(2, 2, 1)
sns.lineplot(x='Year', y='log_hg/ha_yield', data=N_cleaned_df, marker='o')
plt.title('Trend of Log-Transformed Crop Yield Over Time')
plt.xlabel('Year')
plt.ylabel('Log-Transformed Crop Yield (log(hg/ha))')

# Original data for average rainfall
plt.subplot(2, 2, 2)
sns.lineplot(x='Year', y='average_rain_fall_mm_per_year', data=N_cleaned_df, marker='o', color='blue')
plt.title('Trend of Average Rainfall Over Time')
plt.xlabel('Year')
plt.ylabel('Average Rainfall (mm)')

# Original data for average temperature
plt.subplot(2, 2, 3)
sns.lineplot(x='Year', y='avg_temp', data=N_cleaned_df, marker='o', color='green')
plt.title('Trend of Average Temperature Over Time')
plt.xlabel('Year')
plt.ylabel('Average Temperature (°C)')

# Using the cleaned and transformed data for pesticide tonnes
plt.subplot(2, 2, 4)
sns.lineplot(x='Year', y='pesticides_tonnes', data=N_cleaned_df, marker='o', color='red')
plt.title('Trend of Pesticides Tonnes Over Time')
plt.xlabel('Year')
plt.ylabel('Pesticides (tonnes)')

plt.tight_layout()
plt.show()

Rainfall and Temperature: These variables show their trends over time, which are crucial for understanding environmental impacts on crop production.
Pesticides: The trend in pesticide usage over time can correlate with changes in agricultural practices and technological advancements in farming.

In [ ]:
N_cleaned_df.index.unique

In [ ]:
N_cleaned_df.set_index('Year', inplace=True)

In [ ]:
N_cleaned_df.info()

In [ ]:
# Print current frequency
print("Current frequency:", N_cleaned_df.index.freq)

In [ ]:
import pandas as pd
import numpy as np

# Check if 'Year' is the index
if 'Year' not in N_cleaned_df.columns:
    N_cleaned_df.reset_index(inplace=True)

# Convert 'Year' to a datetime type if necessary
if not np.issubdtype(N_cleaned_df['Year'].dtype, np.datetime64):
    N_cleaned_df['Year'] = pd.to_datetime(N_cleaned_df['Year'], format='%Y')

# Set 'Year' as the index
N_cleaned_df.set_index('Year', inplace=True)

# Ensure current frequency is checked
print("Current frequency:", N_cleaned_df.index.freq)

# Aggregate data annually by taking the mean for numeric columns only
numeric_columns = N_cleaned_df.select_dtypes(include=[np.number]).columns
annual_data = N_cleaned_df[numeric_columns].groupby(N_cleaned_df.index.year).mean()

# Creating a new DateTimeIndex with annual start frequency
new_index = pd.date_range(start=f"{annual_data.index.min()}-01-01", periods=len(annual_data), freq='AS')

# Assign the new index to the DataFrame
annual_data.index = new_index

# Verify the data
print(annual_data.head())



In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Perform seasonal decomposition
decomposition = seasonal_decompose(annual_data['log_hg/ha_yield'], model='additive')

# Plotting the decomposed components
fig = decomposition.plot()
fig.set_size_inches(14, 7)
plt.show()


trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(10, 5))
plt.plot(trend, label='Trend')
plt.legend()
plt.show()


### checking for stationarity

In [ ]:
import matplotlib.pyplot as plt

# Calculate the rolling mean and rolling standard deviation
rolling_mean = annual_data['log_hg/ha_yield'].rolling(window=12).mean()  # The window size can be adjusted
rolling_std = annual_data['log_hg/ha_yield'].rolling(window=12).std()

# Plot the original data, the rolling mean, and the rolling standard deviation
plt.figure(figsize=(14, 7))
plt.plot(annual_data['log_hg/ha_yield'], color='blue', label='Original')
plt.plot(rolling_mean, color='red', label='Rolling Mean')
plt.plot(rolling_std, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform the Augmented Dickey-Fuller test to check for stationarity
result = adfuller(annual_data['log_hg/ha_yield'])

# Output the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


ADF Statistic:
−
2.2698
−2.2698, which is greater than any of the critical values at common significance levels (1%, 5%, and 10%). This means we fail to reject the null hypothesis of the ADF test, which states that there is a unit root present in the series.
p-value:
0.1819
0.1819, which is above 0.05, suggesting that we cannot reject the null hypothesis at the 5% significance level. This indicates that the series may have a unit root, implying it is non-stationary.
Critical Values: The test statistic is not lower than any of these, reinforcing the non-rejection of the null hypothesis.

In [ ]:
from scipy.stats import boxcox
import matplotlib.pyplot as plt
import numpy as np

# Apply the Box-Cox Transformation to a specific column
data = cleaned_df['hg/ha_yield']  # Make sure this column has no zero or negative values
data_transformed, lambda_ = boxcox(data)

# Define the inverse Box-Cox transformation
def inv_boxcox(y, lmbda):
    if lmbda == 0:
        return np.exp(y)
    else:
        return np.exp(np.log(lmbda * y + 1) / lmbda)

# You can also transform the data back if needed using the lambda value obtained
data_original = inv_boxcox(data_transformed, lambda_)

# Plotting the original and transformed data for comparison
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data, bins=30, color='blue', alpha=0.7)
plt.title('Original Data')
plt.xlabel('Values')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(data_transformed, bins=30, color='green', alpha=0.7)
plt.title('Box-Cox Transformed Data')
plt.xlabel('Transformed Values')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

print("Lambda used for Box-Cox Transformation:", lambda_)



In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform the Augmented Dickey-Fuller test on the Box-Cox transformed data
adf_result = adfuller(data_transformed)

print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
    print('\t%s: %.3f' % (key, value))

# Evaluate the p-value and ADF Statistic against critical values
if adf_result[1] < 0.05:
    print("The series is stationary.")
else:
    print("The series is not stationary and may need further differencing or transformations.")


### Autocorrelation Function (ACF) & Partial Autocorrelation Function (PACF)

Autocorrelation Function (ACF):
Measures the correlation between time series observations at different lags.

Partial Autocorrelation Function (PACF):
Measures the correlation between observations at different lags while controlling for the values of the intermediate lags.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib.pylab import rcParams

# Set the plot size and style for better visibility
rcParams['figure.figsize'] = (14, 10)  # Increase plot size


# Plotting ACF with adjusted alpha for a clearer view of the confidence interval
plt.figure()
plot_acf(data_transformed, lags=10, alpha=0.05)
plt.title('Autocorrelation Function')

# Plotting PACF with adjusted alpha
plt.figure()
plot_pacf(data_transformed, lags=10, alpha=0.05, method='ywm')
plt.title('Partial Autocorrelation Function')

plt.show()


### Modelling the Time Series

### AutoRegressive (AR) Model

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# ARIMA(p,d,q) where p=2, d=0, q=0
model = ARIMA(data_transformed, order=(2,0,0))
results = model.fit()

# Print the summary of the model
print(results.summary())



No. Observations: 25,874 observations were used in this model, indicating a large dataset.

Model: ARIMA(2, 0, 0) suggests that this is an AutoRegressive (AR) model with 2 lags (p=2), no differencing (d=0), and no moving average components (q=0).

Log Likelihood: -30,656.324 - This value indicates the log likelihood of the model, where higher values are better. In your case, the value is negative and quite large, suggesting potential overfitting or a poor model fit due to the complexity or noise in the data.

AIC: 61,320.649 - The Akaike Information Criterion helps compare different models. Lower AIC values are better. Given the size of your AIC, it might suggest that the model could be improved either by simplifying or by better parameter tuning.

BIC: 61,353.293 - The Bayesian Information Criterion, like AIC, helps in model comparison. It usually penalizes free parameters more strongly than AIC. The high value here again suggests room for model improvement.

Covariance Type: 'opg' suggests that the 'outer product of gradients' method was used for covariance matrix calculation.

Coefficients:
const: The constant term is 9.9497, with a very tight confidence interval, indicating a strong and significant baseline level of the transformed yield data.
ar.L1: The coefficient for the first lag (0.4411) is significant (P>|z| near 0), indicating a strong positive correlation with the previous value in the series.
ar.L2: The coefficient for the second lag (0.1642) is also significant and positive, showing that the second past value also influences the current value but to a lesser extent than the first lag.
sigma2: The variance of the residuals is 0.6261, suggesting the average squared deviations from the predicted values are relatively low, indicating a good fit of the model to the data.

Diagnostic Tests:
Ljung-Box (L1) (Q): A p-value of 0.01 suggests that there may still be some autocorrelation at lag 1 that the model does not account for.
Jarque-Bera (JB): A p-value close to 0 indicates that the residuals of the model do not follow a normal distribution.
Heteroskedasticity (H): The test suggests that the variance of the residuals is not constant. The p-value close to 0 indicates that the assumption of homoscedasticity (constant variance) is violated.

### MA Model (Moving Average Model)

An MA model is another type of time series model that uses dependency between an observation and a residual error from a moving average model applied to lagged observations.

In [ ]:
# Set the order (q)
q = 1

# Create and fit the MA model
ma_model = ARIMA(data_transformed, order=(2,0,q))
ma_results = ma_model.fit()

# Print the summary of the MA model
print(ma_results.summary())


Log Likelihood: -226.336. This value indicates the likelihood of the model given the data, where higher values (closer to zero) generally indicate a better fit.

AIC (Akaike Information Criterion): 458.672. This metric helps to compare models; lower values suggest a better model fit considering the number of parameters used.

BIC (Bayesian Information Criterion): 462.078. This is another criterion for model comparison that penalizes free parameters more strongly than AIC.

const: The estimated constant term is approximately 76,870, suggesting the baseline level of hg/ha_yield in the absence of other influences.

ma.L1: The coefficient for the first lag of the moving average term is 0.9993. This indicates a near-complete carryover of last year's forecast error into this year's prediction, which suggests that the previous year's error almost entirely influences the current year's forecast.

Ljung-Box Test (Q): With a statistic of 13.45 and a p-value near zero, this suggests that there are still autocorrelations in the residuals not captured by the model.

Jarque-Bera (JB): This test checks the normality of residuals. The JB statistic of 1.53 and a p-value of 0.47 do not provide strong evidence against the normality assumption.

Interpretation:

While the MA model captures some of the error dynamics from one year to the next, the significant Ljung-Box statistic suggests that additional lags or perhaps a combination of autoregressive and moving average terms might be necessary to adequately model the data. The near-unit root in the MA term suggests that the series might benefit from additional differencing or a model that captures longer-term trends or cyclical behavior.

### AutoRegressive Moving Average (ARMA) Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Setting the AR and MA components
p = 2
q = 2

# Creating and fitting the ARMA model (ARIMA with d=0)
arma_model = ARIMA(data_transformed, order=(p, 0, q))
arma_results = arma_model.fit()

# Print the summary of the model
print(arma_results.summary())


Model: ARIMA(2, 0, 2) indicates two AR terms and two MA terms without differencing.
Sample: 25,874 observations, indicating a substantial amount of data for the time series analysis.
Covariance Type: 'opg' indicates that the outer product of gradients method was used for estimation.


Log Likelihood: -30,241.695, which provides a measure of the model's goodness of fit; higher values (less negative) are better.
AIC (Akaike Information Criterion): 60,495.391 helps compare models; lower values suggest a better model relative to others by balancing goodness of fit and complexity.
BIC (Bayesian Information Criterion): 60,544.357, similar to AIC, but with a higher penalty for models with more parameters.


const: The model's constant is approximately 9.9496, which could represent the series' mean level after considering the AR and MA components.
ar.L1 and ar.L2: Coefficients for the first and second lags of the AR component. ar.L1 is -0.2064, suggesting a negative influence from the first lag on the current value. ar.L2 is 0.7733, indicating a strong positive effect from the second lag.
ma.L1 and ma.L2: Coefficients for the MA terms. ma.L1 is 0.6154, showing a positive adjustment based on the first lag's error, and ma.L2 is -0.3396, indicating a negative adjustment based on the second lag's error.


Ljung-Box Test: The Q statistic at lag 1 is 23.81 with a p-value of 0.00, suggesting significant autocorrelation in residuals at lag 1.
Jarque-Bera (JB) Test: A JB statistic of 89.34 with a p-value of 0.00 indicates that residuals do not follow a normal distribution.
Heteroskedasticity (H): A heteroskedasticity statistic of 1.51 with a p-value of 0.00 suggests variance inconsistency in the residuals over time.
Skew and Kurtosis: The skewness of 0.05 and kurtosis of 3.27 suggest slight asymmetry and a peak near the normal distribution.


Summary:
The ARIMA(2, 0, 2) model seems to fit the data adequately, with significant parameters indicating a complex interplay of autoregressive and moving average components. However, the diagnostics highlight potential issues with autocorrelation and non-normal distribution of residuals, suggesting further refinement or additional diagnostic checks may be necessary to fully validate the model's assumptions and effectiveness.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Setting the AR and MA components
p = 3
q = 3

# Creating and fitting the ARMA model (ARIMA with d=0)
arma_model = ARIMA(data_transformed, order=(p, 0, q))
arma_results = arma_model.fit()

# Print the summary of the model
print(arma_results.summary())



Model: ARIMA(3, 0, 3) indicating three AR terms and three MA terms.
Sample Size: 25,874 observations, providing a substantial dataset for robust statistical analysis.
Covariance Type: 'opg' indicating the use of the outer product of gradient estimator for covariance computation.
Statistical Performance:
Log Likelihood: -30,013.905, indicating the model's likelihood given the data. Higher (less negative) values indicate better model fit.
AIC (Akaike Information Criterion): 60,043.809, which helps to compare models; lower values generally indicate a better fit considering the complexity and number of parameters.
BIC (Bayesian Information Criterion): 60,109.097, similar to AIC but with a stronger penalty for more complex models, useful for model selection.
Parameter Estimates:
Constant: The constant term (const) approximates 9.9477, suggesting the baseline level around which the series fluctuates.
AR and MA Coefficients:
ar.L1: Coefficient for the first AR term is 0.5702, indicating a moderate positive influence from the first lag.
ar.L2: Coefficient for the second AR term is 0.9741, showing a strong positive influence from the second lag.
ar.L3: Coefficient for the third AR term is -0.5580, suggesting a significant negative impact from the third lag.
ma.L1: Coefficient for the first MA term is -0.1480, a slight negative influence reflecting the error of the first lag.
ma.L2: Coefficient for the second MA term is -0.9139, indicating a strong negative adjustment based on the second lag's error.
ma.L3: Coefficient for the third MA term is 0.1497, showing a modest positive adjustment based on the third lag's error.
Diagnostics:
Ljung-Box Test (Q): A p-value of 0.24 at lag 1 indicates there is no significant autocorrelation in the residuals, which is good as it suggests the model adequately captures the correlation in the data.
Jarque-Bera (JB) Test: A statistic of 79.12 with a p-value near zero indicates that residuals do not follow a normal distribution, suggesting potential issues with the model assumptions.
Heteroskedasticity (H): A test value of 1.52 with a p-value near zero suggests the presence of heteroskedasticity, meaning the variance of residuals is not constant across the series.
Skew and Kurtosis: Skew of 0.11 and kurtosis of 3.16 suggest slight asymmetry and a distribution peak near the normal.
Conclusion:
The ARIMA(3, 0, 3) model demonstrates significant predictive power given its statistical measures and coefficients. However, issues such as non-normal residuals and potential heteroskedasticity should be addressed, possibly through further model refinement or additional diagnostic checking.

Additional Steps:
Collect More Data: If possible, enhance your dataset with more variables that could affect crop yields, such as soil quality data, crop variety information, and more detailed local climate data.

Stakeholder Engagement: Work with local farmers, agricultural experts, and climatologists to refine your models and recommendations to ensure they are practical and grounded in local realities.

**Now we deploy the model**

In [ ]:
!pip install ipywidgets


In [ ]:
print(preprocessor.named_transformers_['cat'].categories_)


In [ ]:
def on_button_click(b):
    input_data = {
        'avg_temp': [inputs['avg_temp'].value],
        'average_rain_fall_mm_per_year': [inputs['average_rain_fall_mm_per_year'].value],
        'pesticides_tonnes': [inputs['pesticides_tonnes'].value],
        'Area': ['Argentina'],  # Use a valid area from your categories
        'Item': ['Maize']  # Use a valid item from your categories
    }

    input_df = pd.DataFrame(input_data)

    # Preprocess the input data
    try:
        preprocessed_data = preprocessor.transform(input_df)

        # Make prediction
        prediction = linear_model.predict(preprocessed_data)

        # Display the prediction
        output.clear_output()
        with output:
            display(f"Predicted Yield: {prediction[0]:.2f} log(hg/ha)")
    except ValueError as e:
        output.clear_output()
        with output:
            display(f"Error: {e}")

button.on_click(on_button_click)

# Display all widgets
display(widgets.VBox([*inputs.values(), button, output]))


In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Define input widgets
inputs = {
    'avg_temp': widgets.FloatText(description='avg_temp', value=25),
    'average_rain_fall_mm_per_year': widgets.FloatText(description='average_rain_fall_mm_per_year', value=800),
    'pesticides_tonnes': widgets.FloatText(description='pesticides_tonnes', value=100),
    'Area': widgets.Dropdown(description='Area', options=['Argentina', 'Australia', 'Brazil', 'Canada', 'China']),
    'Item': widgets.Dropdown(description='Item', options=['Maize', 'Potatoes', 'Wheat', 'Soybeans', 'Rice, paddy'])
}

# Button to trigger the prediction
button = widgets.Button(description="Predict")

# Output widget to display the prediction
output = widgets.Output()

def on_button_click(b):
    input_data = {
        'avg_temp': [inputs['avg_temp'].value],
        'average_rain_fall_mm_per_year': [inputs['average_rain_fall_mm_per_year'].value],
        'pesticides_tonnes': [inputs['pesticides_tonnes'].value],
        'Area': [inputs['Area'].value],
        'Item': [inputs['Item'].value]
    }

    input_df = pd.DataFrame(input_data)

    # Preprocess the input data
    try:
        preprocessed_data = preprocessor.transform(input_df)

        # Make prediction
        prediction = linear_model.predict(preprocessed_data)

        # Display the prediction
        output.clear_output()
        with output:
            display(f"Predicted Yield: {prediction[0]:.2f} log(hg/ha)")
    except ValueError as e:
        output.clear_output()
        with output:
            display(f"Error: {e}")

button.on_click(on_button_click)

# Display all widgets
display(widgets.VBox([*inputs.values(), button, output]))





